S&P500を

(1) 買ってずっとホールド

(2) 一日のOpenで買い、
もし、その日のうちに1%以上上がればCloseで売り
もし、その日のうちに下がればホールド

という2つの戦略をとったときのシミュレーションをやりたい。売買手数料は代金の0.5%とする

In [5]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

# データ取得（S&P500の過去のデータ）
data = yf.download('^GSPC', start='2010-01-01', end='2024-12-31')
data = data[['Open', 'Close']]

# 売買手数料
transaction_fee_rate = 0.005

# (1) 買ってホールド戦略
buy_and_hold = data['Close'] / data['Close'].iloc[0]  # 累積リターン

[*********************100%***********************]  1 of 1 completed


In [4]:
# (2) 条件付き売買戦略
strategy = pd.DataFrame(index=data.index)
strategy['Daily Change (%)'] = (data['Close'] - data['Open']) / data['Open'] * 100
strategy['Position'] = 0.0

# 売買のロジック
position = 0.0
cash = 1.0  # 初期資金 1単位
positions = []
for i in range(len(data)):
    if position == 0:  # 保有なし
        position = cash / data['Open'].iloc[i] * (1 - transaction_fee_rate)  # 購入後の株数
        cash = 0.0
    if strategy['Daily Change (%)'].iloc[i] > 1:  # 1%以上上昇
        cash = position * data['Close'].iloc[i] * (1 - transaction_fee_rate)  # 売却後の現金
        position = 0.0
    positions.append(cash + (position * data['Close'].iloc[i]))

strategy['Position'] = positions
strategy['Strategy Return'] = strategy['Position'] / strategy['Position'].iloc[0]

# 結果のプロット
plt.figure(figsize=(12, 6))
plt.plot(buy_and_hold, label='Buy and Hold', linestyle='--')
plt.plot(strategy['Strategy Return'], label='Conditional Trading Strategy')
plt.title('S&P 500 Strategy Simulation')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid()
plt.show()


[*********************100%***********************]  1 of 1 completed


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().